# ETF flow decomposition

> Nearest neighbor functions, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp flow_decomposition

In [ ]:
# | hide
from nbdev.showdoc import *
from lobster_tools.preprocessing import *
from lobster_tools.querying import *

In [ ]:
# | export
from functools import partial
from itertools import product
from typing import Literal, Optional, get_args

import numpy as np
import pandas as pd
from numpy.typing import NDArray
from sklearn.neighbors import KDTree
import datetime

In [ ]:
# | export
tolerances = ["500us", "1ms"]
resample_freq = "5T"
equities = ["AIG", "AAPL"]
etfs = ["WMT", "GE"]
date_range = "2019-01-02"

In [ ]:
# | export
def get_times(df: pd.DataFrame) -> NDArray[np.datetime64]:
    #TODO better way to rewrite this?
    if df.index.values.dtype == "datetime64[ns]":
        return df.index.values.reshape(-1, 1)
    if "datetime" not in df.columns:
        raise ValueError("No datetime column found in df, or datetimes in index")
    if df["datetime"].dtype != "datetime64[ns]":
        raise ValueError("datetime column is not datetime64[ns] dtype")
    else:
        return df["datetime"].values.reshape(-1, 1)

def str_to_time(time: str, convert_to: str) -> int:
    return pd.Timedelta(time) / pd.Timedelta(1, unit=convert_to)


# TODO: find a better way to compose without lambda
str_to_nanoseconds = lambda x: int(str_to_time(x, convert_to="ns"))


In [ ]:
# | export
def add_neighbors(
    etf_executions: pd.DataFrame,
    equity_executions: pd.DataFrame,
    tolerance: str | list[str],
):
    """Annotate the etf execution dataframe with the indices of the neighbouring equity executions.
    Note: Building the KDTree on the equity dataframe.
    """
    # new addition so it's not inplace    
    etf_executions = etf_executions.copy()

    etf_executions_times = get_times(etf_executions)
    equity_executions_times = get_times(equity_executions)
    equity_tree = KDTree(equity_executions_times, metric="l1")

    def _add_neighbors_col(etf_executions, tolerance_str):
        tolerance_in_nanoseconds = str_to_nanoseconds(tolerance_str)
        etf_executions[f"_{tolerance_str}_neighbors"] = equity_tree.query_radius(etf_executions_times, r=tolerance_in_nanoseconds)

        etf_executions[f"_{tolerance_str}_neighbors"] = etf_executions[f"_{tolerance_str}_neighbors"].apply(
            lambda x: None if x.size == 0 else x
        )

    if isinstance(tolerance, str):
        _add_neighbors_col(etf_executions, tolerance)

    elif isinstance(tolerance, list):
        for tolerance in tolerance:
            _add_neighbors_col(etf_executions, tolerance)
    else:
        raise ValueError("tolerance_str must be a string or a list of strings")
    
    return etf_executions


def drop_all_neighbor_cols(df: pd.DataFrame):
    "Drop neighbor columns inplace."
    neighbor_column_names = df.filter(regex="neighbors").columns
    df.drop(columns=neighbor_column_names, inplace=True)


In [ ]:
# | export
def col_to_dtype_inputing_mapping(col, col_to_dtype_dict):
    for k, v in col_to_dtype_dict.items():
        if k in col:
            return v


col_to_dtype = partial(
    col_to_dtype_inputing_mapping,
    col_to_dtype_dict={
        "notional": pd.SparseDtype(float, 0),
        "num_trades": pd.SparseDtype(int, 0),
        "distinct_tickers": pd.SparseDtype(int, 0),
    },
)

In [ ]:
# | export
features = ["distinct_tickers", "notional", "num_trades"]
all_index = ["_".join(t) for t in product(features, ["ss", "os"], ["bf", "af"])]

empty_series = pd.Series(index=all_index, dtype="Sparse[float]").fillna(0)
empty_series = pd.Series(index=all_index, dtype="float").fillna(0)


def multi_index_to_single_index(df: pd.DataFrame) -> pd.DataFrame:
    df.index = ["_".join(index_tuple) for index_tuple in df.index]
    return df


def groupby_index_to_series(df: pd.DataFrame) -> pd.Series:
    """Hierachical groupby index with one column to flattened series. Prepending the column name to the index."""
    return df.stack().reorder_levels([-1, 0, 1]).pipe(multi_index_to_single_index)


def compute_features(
    etf_trade_time, etf_trade_direction, neigh: Optional[np.ndarray], equity_executions: pd.DataFrame,
) -> pd.DataFrame:
    # NOTE: filtering didn't work out great

    if neigh is None:
        # is this the right thing to do
        return empty_series
    elif isinstance(neigh, np.ndarray):
        df = equity_executions.iloc[neigh].assign(
            bf_af=lambda df: df.index < etf_trade_time,
            ss_os=lambda df: df.direction == etf_trade_direction,
        )
        df["ss_os"] = (
            df["ss_os"].apply(lambda x: "ss" if x else "os").astype("category")
        )
        df["bf_af"] = (
            df["bf_af"].apply(lambda x: "bf" if x else "af").astype("category")
        )

        df_subset = df[["ticker", "ss_os", "bf_af", "price", "size"]]

        # TODO: think if there's a way to do one chained line. I'm grouping on three vs two..hmm

        # notional value and num trades
        notional_and_num_trades = (
            df_subset.eval('notional = price * size.astype("int64")')
            .groupby(["ss_os", "bf_af"])
            .agg(notional=("notional", "sum"), num_trades=("size", "count"))
            .pipe(groupby_index_to_series)
        )

        # distinct tickers
        distinct_tickers = (
            df_subset.drop(columns="size")
            .groupby(["ticker", "ss_os", "bf_af"])
            .count()
            .applymap(lambda x: x > 0)
            .groupby(["ss_os", "bf_af"])
            .sum()
            .rename(columns={"price": "distinct_tickers"})
            .pipe(groupby_index_to_series)
        )

        # print(pd.concat([notional_and_num_trades, distinct_tickers]))
        return (
            pd.concat([notional_and_num_trades, distinct_tickers])
            .reindex(all_index)
            .fillna(0)
        )
    else:
        raise ValueError("neigh must be None or list")


In [ ]:
# | export
def append_features(etf_executions: pd.DataFrame, equity_executions: pd.DataFrame) -> pd.DataFrame:
    "Note that this function is not inplace."
    # infer tolerances from column names
    column_names = etf_executions.filter(regex="neighbors").columns.values.tolist()
    tolerances = [i.split("_")[1] for i in column_names]
    
    # TODO: check if its faster to partial compute features with equity executions
    
    features_dfs = []
    for tolerance in tolerances:
        # add_neighbors(df, equity_executions, tolerance)
        features = etf_executions.apply(
            lambda row: compute_features(
                row.name, row.direction, row[f"_{tolerance}_neighbors"], equity_executions=equity_executions,
            ),
            axis=1,
            result_type="expand",
        ).add_prefix(f"_{tolerance}_")

        features = features.astype({col: col_to_dtype(col) for col in features.columns})

        features_dfs += [features]

    features_df = pd.concat(features_dfs, axis=1)
    return pd.concat([etf_executions, features_df], axis=1)


In [ ]:
# | export
def count_non_null(df, tolerance):
    return df[f"_{tolerance}_neighbors"].notnull().sum()

In [ ]:
# | export
def marginalise(df: pd.DataFrame, 
                over: Literal["same_sign/opposite_sign", "before/after"], # which feature to marginalise over 
                drop=False # drop or keep columns that were marginalised over
                ) -> pd.DataFrame:
    """Marginalise over a specific feature split by summing columns together. Compute marginalised features from dataframe by summing over columns with keywords."""
    
    # TODO think whether inplace is better
    df = df.copy()

    match over:
        case "same_sign/opposite_sign":
            keyword0, keyword1 = "_ss", "_os"
        case "before/after":
            keyword0, keyword1 = "_bf", "_af"
        case _:
            # TODO figure out how to do: f"over must be keywords must be one of {get_args(over)}"
            raise ValueError(f"invalid 'over' keyword")
    
    for col_name in df.columns:
        if keyword0 in col_name:
            base_col_name = col_name.replace(keyword0, "")
            opposite_col_name = col_name.replace(keyword0, keyword1)
            df[base_col_name] = df[col_name] + df[opposite_col_name]
            
        # TODO sanity check this
            if drop:
                df.drop(columns=[col_name, opposite_col_name], inplace=True)
    
    return df

In [ ]:
# |export
aggregate_before_after = partial(marginalise, over="before/after", drop=False)
aggregate_same_sign_opposite_sign = partial(marginalise, over="same_sign/opposite_sign", drop=False)

In [ ]:
# |export
def add_arb_tag(df: pd.DataFrame, tag_functions: list[str]) -> None:
    """Add arb tags using a list of str eval expressions."""
    for tag_function in tag_functions:
        df.eval(tag_function, inplace=True)
    return None


# manually specify tag function strings
# tag_functions = [
#     "_500ms_arb_tag = _500ms_notional_os > 0",
#     "_300ms_arb_tag = _300ms_notional_os > 0",
# ]

# # generate tag function strings
# tag_functions = [
#     f"_{tolerance}_arb_tag = _{tolerance}_notional_os > 0" for tolerance in tolerances
# ]

In [ ]:
# | export
def drop_features(df: pd.DataFrame) -> None:
    """Drops all intermediate features, and just leaves the arbitrage tags.
    Not the nicest way. Could do better regex."""
    features_and_arb_tag = set(df.filter(regex="^_[0-9]+ms_").columns)
    arb_tag = set(df.filter(regex="arb_tag").columns)
    features = features_and_arb_tag - arb_tag
    df.drop(columns=features, inplace=True)
    return None

In [ ]:
# | export
def split_isolated_non_isolated(etf_executions: pd.DataFrame, tolerance) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Returns a tuple of (isolated, non_isolated). For now, use deep copy, although this may not be great."""
    tolerance_str = f"_{tolerance}_neighbors"
    isolated_indices = etf_executions[tolerance_str].isna()
    return etf_executions[isolated_indices].copy(deep=True), etf_executions[~isolated_indices].copy(deep=True)

In [ ]:
# | export
def ofi(df: pd.DataFrame, resample_freq: str = "5T", suffix: str | None = None) -> pd.DataFrame:
    suffix = "" if suffix is None else ("_" + suffix)
    
    ofi_series = []
    for ticker in df.ticker.unique().tolist():
        ofi = (
            df.query("ticker == @ticker")
            .eval('signed_size = size.astype("int64") * direction')[
                ["size", "signed_size"]
            ]
            .resample(resample_freq, label="right")
            .sum()
            .eval("ofi = signed_size / size")
            .ofi.rename(f"_{ticker}_{resample_freq}_ofi" + suffix)
            .fillna(0)
        )
        ofi_series += [ofi]

    ofi_df = pd.concat(ofi_series, axis=1)
    return ofi_df

In [ ]:
# | export
def ofis(df: pd.DataFrame, # dataframe that contains only one etf (i.e has already been queried for one etf)
        resample_freq: str = "5T", suffix: str | None = None) -> pd.DataFrame:
    """DIFF Function"""
    suffix = "" if suffix is None else ("_" + suffix)
    
    ofi_series = []
    etfs = df.ticker.unique().tolist()
    for etf in etfs:
        
        ofi = (
            df.query("ticker == @etf")
            .eval('signed_size = size.astype("int64") * direction')[
                ["size", "signed_size"]
            ]
            .resample(resample_freq, label="right")
            .sum()
            .eval("ofi = signed_size / size")
            .ofi.rename(f"_{resample_freq}_{etf}_ofi" + suffix)
            .fillna(0)
        )
        ofi_series += [ofi]

    ofi_df = pd.concat(ofi_series, axis=1)
    return ofi_df

In [ ]:
# | export
def not_ofi(df: pd.DataFrame, resample_freq: str = "5T", suffix: str | None = None) -> pd.DataFrame:
    """Quick modification where I don't normalize by size."""
    suffix = "" if suffix is None else ("_" + suffix)
    
    ofi_series = []
    etfs = df.ticker.unique().tolist()
    for etf in etfs:
        ofi = (
            df.query("ticker == @etf")
            .eval('signed_size = size.astype("int64") * direction')[
                ["size", "signed_size"]
            ]
            .resample(resample_freq, label="right")
            .sum()
            .signed_size.rename(f"_{resample_freq}_{etf}_aggregated_size" + suffix)
            .fillna(0)
        )
        ofi_series += [ofi]

    ofi_df = pd.concat(ofi_series, axis=1)
    return ofi_df

In [ ]:
# | export

# TODO: extend function / create another functino to loop over sampling freq
# TODO: etfs is a global variable in this function. compute it within the function or pass it as an argument


def compute_ofi(
    df: pd.DataFrame, resample_freq: str = "5T", arb_filter: Optional[str] = None
) -> pd.DataFrame:
    """Computes OFI by removing the arbitrage flows and resampling to 5 minutes."""
    match arb_filter:
        case "only_arb":
            df = df.query("arb_tag == True")
            arb_filter_str = "_only_arb"
        case "no_arb":
            df = df.query("arb_tag == False")
            arb_filter_str = "_no_arb"
        case None:
            arb_filter_str = ""

    ofi_series = []
    for etf in etfs:
        ofi = (
            df.query("ticker == @etf")
            .eval('signed_size = size.astype("int64") * direction')[
                ["size", "signed_size"]
            ]
            .resample(resample_freq)
            .sum()
            # .apply(lambda df: df.signed_size / df.size, axis=1)
            .eval("ofi = signed_size / size")
            .ofi.rename(f"_{resample_freq}_{etf}_ofi" + arb_filter_str)
            .fillna(0)
        )
        ofi_series += [ofi]

    ofi_df = pd.concat(ofi_series, axis=1)
    return ofi_df



In [ ]:
directory = "/home/petit/Documents/data/lobster/csv"
tolerances = ["500us", "1ms"]
resample_freq = "5T"
equities = ["AIG", "AAPL"]
etfs = ["WMT", "GE"]
date_range = "2019-01-02"

In [ ]:
load_lobster = partial(load_lobster, directory_path="/home/petit/Documents/data/lobster/csv")

In [ ]:
# equity_lobster = load_lobster(date_range=date_range, ticker=equities)
# etf_executions = load_etf_executions(date_range=date_range, tickers=etfs)

In [ ]:
# | export
def resample_mid(df: pd.DataFrame, resample_freq="5T"):
    return df.resample(resample_freq, label="right").last().eval("mid = bid_price_1 + (ask_price_1 - bid_price_1) / 2")["mid"]

In [ ]:
# | export
def restrict_common_index(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """Restrict two dataframes to their common index."""
    common_index = df1.index.intersection(df2.index)
    return df1.loc[common_index], df2.loc[common_index]

In [ ]:
# | export
def markout_returns(
    df,  # dataframe to infer times to markout from
    markouts: list[str],  # list of markouts to compute returns for
) -> pd.DataFrame:
    # TODO: markouts of string literals also?
    return pd.DataFrame(index=df.index, data={f"_{markout}": df.index + pd.Timedelta(markout) for markout in markouts})

In [ ]:
# # | export
# def clip_for_markout(df, max_markout):
#     end = max(df.index) - pd.Timedelta(max_markout)
#     end = end.time()
#     return clip_times(df, end=end)

In [ ]:
# | export

# TODO fix this
# def compute_returns():
#     index = clip_for_markout(etf_executions.resample(resample_freq, label="right").last(), max_markout=max_markout).index

#     returns = {}
#     for ticker in etfs:
#         df = pd.DataFrame(index=index)
#         for markout in ["0S"] + markouts:
#             df[f"_{markout}"] = mids.loc[df.index + pd.Timedelta(markout), ticker].values

#         for markout in markouts:
#             df.eval(f"return_{markout} = (_{markout} / _0S ) - 1", inplace=True)

#         df["return_contemp"] = mids[ticker].resample("5T").first().pct_change()
#         df_returns = df.filter(regex="return")
#         df_returns.columns = [column.replace("return_", "") for column in df_returns.columns]
#         df_returns.columns = [("_" + column if column[0].isdigit() else column) for column in df_returns.columns ]
#         returns[ticker] = df_returns
#     return returns

# returns = compute_returns()

In [ ]:
# ofi_all = ofi(etf_executions, resample_freq="5T", suffix="all")
# ofi_all

In [ ]:
# | export
# TODO: TEMP FIX. PUSH TO PYPI AND REINSTALL
def clip_df_times(df: pd.DataFrame, start: datetime.time | None = None, end: datetime.time | None = None) -> pd.DataFrame:
        """Clip a dataframe or lobster object to a time range."""
        # TODO: improve this function? with the 4 if statements lol, i guess i could clip the index twice and have two if statements
        if start and end:
            return df.iloc[(df.index.time >= start) & (df.index.time < end)]
        elif start:
            return df.iloc[df.index.time >= start]
        elif end:
            return df.iloc[df.index.time < end]
        else:
            raise ValueError("start and end cannot both be None")


In [ ]:
# | export
def clip_for_markout(df, max_markout):
    end = (max(df.index) - pd.Timedelta(max_markout)).time()
    # end = end.time()
    return clip_df_times(df, end=end)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()